# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [22]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/Users/kirillsikamov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/Users/kirillsikamov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [209]:
def minor(M, i, j): #получим минор вырезая строки и столбцы 
    M = np.delete(M, i, 0)
    M = np.delete(M, j, 1) 
    return M  
    
def det(A): #подсчитаем определитель (для квадратной матрицы) по строке
    n = A.shape[0]
    if n == 1:
        return A[0][0]
    sgn = 1
    d = 0
    for j in range( n ):
        d += A[0][j]*sgn*det( minor( A, 0, j ) ) 
        sgn = sgn*(-1) 
    return d

def LU(A): #проверяем на вырожденность миноров лидирующих
    n = A.shape[0]
    for i in range(n):
        x = np.delete((np.delete(A, np.s_[(i+1):N+1], 1)), np.s_[(i+1):N+1], 0)
        if det(x)==0:
            print('разложение не работает')
            break
        elif i == n-1:
            print('разложение работает')
    return


print('проверка для a1:') 
LU(a1)
print('проверка для a:') 
LU(a)

проверка для a1:
разложение не работает
проверка для a:
разложение работает


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

Реализуем следующий алгоритм:

In [218]:
def diy_lu_pivoiting(a): 
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        if u[j, j] == 0: #проверяем не равен ли 0 элемент на диагонали
            b = u.copy() 
            a = u.copy() 
            for i in range(j+1, N ): 
                if u[i,j] != 0: #перестановка
                    b[j] = a[i] 
                    b[i] = a[j]
                    u = b
                    c = P.copy() #дублируем и запоминаем для матрицы перестановки
                    d = P.copy()
                    d[j] = c[i]
                    d[i] = c[j]
                    P = d
                    break
                else: #если не равен 0 то по классике
                    break
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, P

Найдем иссходную матрицу:

In [221]:
L,u,P = diy_lu_pivoiting(a1)

print(P.T@L@u, "\n") 
print(a1, "\n")
print(P.T@L@u-a1, "\n")

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  1.665e-16  5.551e-17]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]] 



Видно, что разность элементов матрицы меньше 10^(-16), что говорит о том что разложение работает